In [2]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta

In [3]:
# 더미 데이터 생성을 위한 Faker모듈 객체 설정
fake = Faker('ko_KR')

In [13]:
num_users = 1
start_date = "2023-01-01"
end_date = "2023-12-31"
output_file= "expenses.csv"

# 입력된 날짜 문자열을 datetime 객체로 변환
start_date = datetime.strptime(start_date, "%Y-%m-%d")
end_date = datetime.strptime(end_date, "%Y-%m-%d")

In [5]:
# 비용 데이터set - 튜플은 금액범위로 설정, 리스트는 값이 금액으로 랜덤 선택

# 월고정비 항목 및 비용 범위
monthly_fixed_costs = {
    "월세": (300000, 5500000), "관리비": (80000, 120000),"전기요금": (20000, 150000),"수도요금": (10000, 30000),
    "가스요금": (10000, 150000),"인터넷비": (20000, 50000),"통신비": (20000, 150000),"보험료": (30000, 300000),
    "교통비": (10000, 100000),"자동차할부금": (250000, 500000),"학원비": (150000, 250000),"x플릭스구독": [8000],
    "x튜브구독": [8000],"음악구독": [5000],"미용실": (20000, 150000),"적금": (10000, 200000)
}

# 연고정비 항목 및 비용 범위
annual_fixed_costs = {
    "자동차보험료": (500000, 1000000),"헬스장": (300000, 500000),"카드연회비": [5000, 10000, 20000, 50000]
}

# 상시비용 (매일 0~2회) 항목 및 비용 범위 설정
daily_expenses_very_high_freq = {
    "xx식당": (6000, 20000), "xx카페": (1000, 10000)
}

# 상시비용 (한달 5~20회) 항목 및 비용 범위 설정
expenses_high_freq = {
    "편의점": (500, 20000)
}

# 상시비용 (한달 약 3회) 항목 및 비용 범위 설정
monthly_expenses_mid_freq = {
    "주유비": (30000, 100000), "택시비": (3800, 50000),"택배비": (3000, 5000), "도서구매": (3000, 30000),
    "영화": [12000, 15000], "생활용품구매": (10000, 50000)
}

# 상시비용 (1년 약 5회) 항목 및 비용 범위 설정
annual_expenses_low_freq = {
    "교육비": (80000, 400000), "병원비": (30000, 70000), "약값": (10000, 30000), "의류비": (20000, 300000),
    "여행비": (500000, 1500000), "가구구매": (300000, 700000), "공연관람": (50000, 100000), "자동차수리": (200000, 400000),
    "화장품구매": (20000, 80000), "건강식품구매": (10000, 50000), "전자제품구매": (500000, 1500000)
}

In [6]:
# 렌덤 추출 함수 - 주어진 비용 딕셔너리에서 랜덤한 비용 항목과 금액을 반환
def random_expense(expense_dict):
    
    # 비용 항목에서 랜덤으로 key값을 뽑고 해당 비용을 value로 설정
    key = random.choice(list(expense_dict.keys()))
    value = expense_dict[key]

    if isinstance(value, tuple):  # value값이 튜플이면,
        return key, random.randint(*value)  # 값 사이의 랜덤한 정수를 생성. *value=튜플의 요소를 개별 인수로 분해

    elif isinstance(value, list):  # value값이 리스트면,
        return key, random.choice(value)  # 값 중에 랜덤으로 1개를 뽑음
    
    else:
        return key, value

In [7]:
# 월의 마지막 일 뽑기 함수
def lastday_of_month(year, month):
    if month == 12:  # 12월일 경우는,
        return 31  # 31일
    
    return (datetime(year, month+1, 1) - timedelta(days=1)).day  # 12월이 아닐경우, (년,다음월,1일-1일).day = 해당월 마지막일

In [16]:
# 비용 생성
expenses = []
for qty in range(num_users):
    
    # 변수 초기화
    user_id = f'user_{qty+1}'  # user_id 생성   
    current_date = start_date
    high_freq_days = []

    # 연간 (1회)고정비용 생성
    for expense in annual_fixed_costs.keys():
        expense_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))  # 소비일 생성
        amount = annual_fixed_costs[expense]  # 해당 expense의 딕셔너리value값을 amount로 설정
        if isinstance(amount, tuple):
            amount = random.randint(*amount)
        elif isinstance(amount, list):
            amount = random.choice(amount)
        expenses.append((user_id, expense_date, expense, amount))

    # 지정된 기간 동안의 소비내역 생성
    while current_date <= end_date:

        # 월별 고정비용 (매월 1일로 지정)
        if current_date == 1:
            for expense, amount in monthly_fixed_costs.items():
                if isinstance(amount, tuple):
                    amount = random.randint(*amount)
                elif isinstance(amount, list):
                    amount = random.choice(amount)
                expenses.append((user_id, expense_date, expense, amount))


        # 빈도 매우높은 상시 비용 (매일 0~2회)
        for qty in range(random.randint(0, 2)):
            expense, amount = random_expense(daily_expenses_very_high_freq)
            expenses.append((user_id, current_date, expense, amount))


        # 빈도 높은 상시 비용 (한달 5~20회)
        if current_date == 1:  # 매달 1일
            lastday = lastday_of_month(current_date.year, current_date.month)  # 월 말일
            high_freq_days = random.sample(range(1, lastday+1), random.randint(5, 20))  # 5~20개의 소비발생날짜 생성

        if current_date.day in high_freq_days:  # 현재 날짜가 생성된 소비발생날짜이면,
            for qty in range(random.randint(0, 3)):  # 1일 0~3번 생성
                expense, amount = random_expense(expenses_high_freq)
                expenses.append((user_id, current_date, expense, amount))


        # 빈도 중간 상시 비용 (한달 3회 정도)
        if random.randint(1, 10) <=3:  # 30%의 확률로 발생여부 설정
            for qty in range(random.randint(1, 2)):  # 1~2가지가 발생할 수 있도록 설정
                expense, amount = random_expense(monthly_expenses_mid_freq)
                expenses.append((user_id, current_date, expense, amount))


        # 빈도 낮은 상시비용 추가 (1년에 10회 정도)
        if random.randint(1, 365) <= 10: # 1년에 5회정도 발생할 수 있도록 설정
            for qty in range(random.randint(1, 2)):
                expense, amount = random_expense(annual_expenses_low_freq)
                expenses.append((user_id, current_date, expense, amount))

        # 날짜 1일 증가
        current_date += timedelta(days=1)

In [17]:
# 데이터프레임 생성
df = pd.DataFrame(expenses, columns=["user_id", "date", "expense", "amount"])
df

,user_id,date,expense,amount
0,user_1,2023-06-10,자동차보험료,846796
1,user_1,2023-06-13,헬스장,330400
2,user_1,2023-01-04,카드연회비,5000
3,user_1,2023-01-01,xx식당,13775
4,user_1,2023-01-01,택배비,4497
...,...,...,...,...
535,user_1,2023-12-27,택배비,3546
536,user_1,2023-12-28,병원비,38679
537,user_1,2023-12-28,가구구매,604486
538,user_1,2023-12-31,도서구매,13544


In [18]:
# 데이터프레임 CSV 파일로 저장
df.to_csv(f'.\data\{output_file}', index=False, encoding='cp949')